# Change Data Capture




#### This notebook implements the CDC process for the Air_Carrier table. Here are the steps in the process:  

1.   Make a copy of the CSV file so we can revert back if needed `air_carriers_copy.csv`

2.   Simulate changes (inserts, updates, and deletes) by manually altering `air_carriers.csv`:

**Inserts**: Added 3 records to the file:<br>
`{30001,"Centrum Air: CAX"}` <br>
`{30002,"City Airlines: CAY"}` <br>
`{30003,"Greater Bay Airlines: GBY"}` <br>

**Updates**: removed the substrings `(1)` and `(2)` from all the records in the file by doing a find/replace. Changed 109 existing records.<br>

**Deletes**: removed the four records starting with the substring `Desert`:<br>
`{19174,"Desert Pacific Airways: DPA"}` <br>
`{19175,"Desert Pacific Airlines Inc.: DPI"}` <br>
`{19176,"Desert Airlines: DST"}` <br>
`{19618,"Desert Sun Airlines: DSA"}` <br><br>

4.   Rename `air_carriers.csv` to `air_carriers_032324.csv`, to indicate the date in which the changes arrived.

5.   Make a new folder in the GCS bucket called `incrementals`.

6.   Copy `air_carriers_032324.csv` into our `incrementals` folder in GCS.

7.   Make a copy of the raw table in case we need to revert back to it (`air_carriers_copy`).

8.   Make a new dataset in BigQuery to hold loading tables (`airline_ldg`).

9.   Load `air_carriers_032324.csv` into the loading area (`airline_ldg.air_carriers_032324`).

10.   Detect the deltas between `airline_ldg.air_carriers_032324` and `airline_raw.air_carriers`:
- If the new record is the same as the old one, ignore it because it means that it's unchanged.
- If the new record is different from the old record, update the old record in the raw table such that it matches the new record in the loading table.
- If the old record in the raw table does not have a corresponding new record in the loading table, delete it from the raw table.  

11.   Make a copy of staging table (`airline_stg.Air_Carrier_copy`) so we can use it for comparison.

12.   Re-create the staging table `airline_stg.Air_Carrier` by applying the same cleansing, modeling, and validation logic as before. The logic in staging remains the same as before.

13.   Make a copy of target table (`airline_csp.Air_Carrier_copy`) so we can use it for comparison.

14.  Once the staging table is ready, merge it into the target table (`airline_csp.Air_Carrier`) as follows:

- If the record in staging is the same as the record in the target table, ignore it.
- If the record in staging is different from the record in the target table, set the `discontinue_time` of the existing record in the target table and insert the new record into the target table. The `effective_time` of the new record should be equal to the current timestamp and the `discontinue_time` of the old record should be equal to the current timestamp - 1 second.
-If a record in the target table does not have a record in staging, set its `discontinue_time` to current timestamp - 1 second.  


The rest of this notebook implements the steps 7-14. Note: steps 1-6 were done by hand.  

### Create backup of the raw table (step 7)

### **Don't re-run this cell after mutating the raw table!**

In [ ]:
%%bigquery
create or replace table airline_raw.air_carriers_copy as
  select * from airline_raw.air_carriers

Query is running:   0%|          |

""


### Create the loading dataset (step 8)

In [ ]:
%%bigquery
create schema if not exists airline_ldg
  options(location="us")

Query is running:   0%|          |

""


### Create and populate the loading table (step 9)

In [6]:
from google.cloud import storage
from google.cloud import bigquery

project_id = "cs329e-sp2024"
bucket_name = "cs329e-open-access"
folder_name = "incrementals"
dataset_name = "airline_ldg"
region = "us"
file_name = 'air_carriers_032324.csv'
table_name = 'air_carriers_032324'

storage_client = storage.Client()
bq_client = bigquery.Client()

schema = [
  bigquery.SchemaField("code", "INTEGER", mode="REQUIRED"),
  bigquery.SchemaField("description", "STRING", mode="REQUIRED"),
  bigquery.SchemaField("load_time", "TIMESTAMP", mode="REQUIRED", default_value_expression="CURRENT_TIMESTAMP"),
]

uri = "gs://{}/{}/{}".format(bucket_name, folder_name, file_name)
table_id = "{}.{}.{}".format(project_id, dataset_name, table_name)

table = bigquery.Table(table_id, schema=schema)
table = bq_client.create_table(table, exists_ok=True)
print("Created table {}".format(table.table_id))

# remove the load_time field from the schema before loading the data,
# the load_time value will be auto-generated
del schema[-1]

job_config = bigquery.LoadJobConfig(
      schema=schema,
      skip_leading_rows=1,
      source_format=bigquery.SourceFormat.CSV,
      write_disposition="WRITE_TRUNCATE",
      field_delimiter=","
    )

load_job = bq_client.load_table_from_uri(uri, table_id, job_config=job_config)
load_job.result()

destination_table = bq_client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))


Created table air_carriers_032324
Loaded 1655 rows.


### Detect the deltas and refresh the raw table (step 10)

#### Identify the deltas

In [7]:
%%bigquery
select t.code, t.description as new_description, r.description as old_description
from airline_ldg.air_carriers_032324 t full join airline_raw.air_carriers r on t.code = r.code
where t.description != r.description
or t.description is null or r.description is null
order by t.code

Query is running:   0%|          |

Downloading:   0%|          |

,code,new_description,old_description
0,<NA>,None,Desert Sun Airlines: DSA
1,<NA>,None,Desert Pacific Airlines Inc.: DPI
2,<NA>,None,Desert Pacific Airways: DPA
3,<NA>,None,Desert Airlines: DST
4,19071,Antilles Air Boats Inc.: AAB,Antilles Air Boats Inc.: AAB (1)
...,...,...,...
111,20440,Lineas Aereas Del Caribe: LC,Lineas Aereas Del Caribe: LC (1)
112,20441,Legend Airlines: LC,Legend Airlines: LC (2)
113,30001,Centrum Air: CAX,None
114,30002,City Airlines: CAY,None


#### Process new records (inserts)

In [8]:
%%bigquery
select (select count(*) from airline_ldg.air_carriers_032324) as ldg_count,
(select count(*) from airline_raw.air_carriers) as raw_count

Query is running:   0%|          |

Downloading:   0%|          |

,ldg_count,raw_count
0,1655,1656


In [9]:
%%bigquery
insert into airline_raw.air_carriers(code, description, load_time)
  (select *
  from airline_ldg.air_carriers_032324
  where code not in (select code from airline_raw.air_carriers))

Query is running:   0%|          |

""


In [10]:
%%bigquery
select (select count(*) from airline_ldg.air_carriers_032324) as ldg_count,
(select count(*) from airline_raw.air_carriers) as raw_count

Query is running:   0%|          |

Downloading:   0%|          |

,ldg_count,raw_count
0,1655,1659


#### Process the changed records (updates)

In [ ]:
%%bigquery
select t.code, t.description as new_description, r.description as old_description
from airline_ldg.air_carriers_032324 t join airline_raw.air_carriers r on t.code = r.code
where t.description != r.description
order by t.code

Query is running:   0%|          |

Downloading:   0%|          |

,code,new_description,old_description
0,19071,Antilles Air Boats Inc.: AAB,Antilles Air Boats Inc.: AAB (1)
1,19083,Air Central Inc. : ACT,Air Central Inc. (1): ACT
2,19385,Reeve Aleutian Airways Inc.: RV,Reeve Aleutian Airways Inc.: RV (1)
3,19391,Pacific Southwest Airlines: PS,Pacific Southwest Airlines: PS (1)
4,19395,Mid-South Aviation Inc. : RCA,Mid-South Aviation Inc. (1): RCA
...,...,...,...
104,20425,Viscount Air Service Inc.: VIQ,Viscount Air Service Inc.: VIQ (1)
105,20435,Frontier Airlines Inc. : FL,Frontier Airlines Inc. (1): FL (1)
106,20438,Air Club International: HB,Air Club International: HB (1)
107,20440,Lineas Aereas Del Caribe: LC,Lineas Aereas Del Caribe: LC (1)


In [11]:
%%bigquery
update airline_raw.air_carriers r
  set r.description = (select l.description from airline_ldg.air_carriers_032324 l where l.code = r.code),
  r.load_time = (select l.load_time from airline_ldg.air_carriers_032324 l where l.code = r.code)
  where r.description != (select description from airline_ldg.air_carriers_032324 l where l.code = r.code)
  and r.code = (select code from airline_ldg.air_carriers_032324 l where l.code = r.code)

Query is running:   0%|          |

""


In [12]:
%%bigquery
select l.code, l.description as new_description, r.description as old_description
from airline_ldg.air_carriers_032324 l join airline_raw.air_carriers r on l.code = r.code
where l.description != r.description
order by l.code

Query is running:   0%|          |

Downloading: |          |

,code,new_description,old_description


#### Process the deleted records (deletes)

In [13]:
%%bigquery
select r.code as old_code, r.description as old_description, l.code as new_code, l.description as new_description
from airline_ldg.air_carriers_032324 l right join airline_raw.air_carriers r on l.code = r.code
where l.code is null
order by r.code

Query is running:   0%|          |

Downloading:   0%|          |

,old_code,old_description,new_code,new_description
0,19174,Desert Pacific Airways: DPA,<NA>,None
1,19175,Desert Pacific Airlines Inc.: DPI,<NA>,None
2,19176,Desert Airlines: DST,<NA>,None
3,19618,Desert Sun Airlines: DSA,<NA>,None


In [14]:
%%bigquery
delete from airline_raw.air_carriers r
where r.code not in (select l.code from airline_ldg.air_carriers_032324 l)

Query is running:   0%|          |

""


In [16]:
%%bigquery
select r.code as old_code, r.description as old_description, l.code as new_code, l.description as new_description
from airline_ldg.air_carriers_032324 l right join airline_raw.air_carriers r on l.code = r.code
where l.code is null
order by r.code

Query is running:   0%|          |

Downloading: |          |

,old_code,old_description,new_code,new_description


In [17]:
%%bigquery
select * from airline_raw.air_carriers
order by load_time desc

Query is running:   0%|          |

Downloading:   0%|          |

,code,description,load_time
0,30003,Greater Bay Airlines: GBY,2024-03-29 16:35:34.693043+00:00
1,30001,Centrum Air: CAX,2024-03-29 16:35:34.693043+00:00
2,30002,City Airlines: CAY,2024-03-29 16:35:34.693043+00:00
3,20072,Scanair: SIQ,2024-03-29 16:35:34.693043+00:00
4,20298,Jes Air Ltd.: JX,2024-03-29 16:35:34.693043+00:00
...,...,...,...
1650,19234,Crown Air: KWZ,2024-01-26 22:23:22.051288+00:00
1651,19256,Metroflight Airlines: MTR,2024-01-26 22:23:22.051288+00:00
1652,19270,Offshore Logistics: OFF,2024-01-26 22:23:22.051288+00:00
1653,21919,Flying Group Lux S.A.: FYQ,2024-01-26 22:23:22.051288+00:00


## Create copy of staging table (step 11)

**Don't re-run this cell after mutating the staging table.**

In [ ]:
%%bigquery
create or replace table airline_stg.Air_Carrier_copy as
  select * from airline_stg.Air_Carrier

Query is running:   0%|          |

""


## Re-create the staging table (step 12)

#### **Note: This logic is the same as before (and was copied from previous notebooks).**

---



In [18]:
%%bigquery
create or replace table airline_stg.Air_Carrier as
  select airline_id, description_array[0] as airline_name, description_array[1] as airline_code, 'bird' as data_source, load_time
  from
  (select code as airline_id, split(description, ':') as description_array, load_time
  from airline_raw.air_carriers)

Query is running:   0%|          |

""


In [19]:
%%bigquery
select (select count(*) from airline_raw.air_carriers) as raw_count,
(select count(*) from airline_stg.Air_Carrier) as staging_count

Query is running:   0%|          |

Downloading:   0%|          |

,raw_count,staging_count
0,1655,1655


In [20]:
%%bigquery
alter table airline_stg.Air_Carrier
  add primary key (airline_id) not enforced;

Query is running:   0%|          |

""


In [21]:
%%bigquery
select airline_id, count(*) duplicate_records
from airline_stg.Air_Carrier
group by airline_id
having count(*) > 1
order by count(*) desc

Query is running:   0%|          |

Downloading: |          |

,airline_id,duplicate_records


## Create copy of target table (step 13)

**Don't re-run this cell after mutating the target table.**

In [ ]:
%%bigquery
create or replace table airline_csp.Air_Carrier_copy as
  select * from airline_csp.Air_Carrier

Query is running:   0%|          |

""


## Merge staging into target table (step 14)

In [22]:
%%bigquery
select count(*) as num_records from airline_csp.Air_Carrier

Query is running:   0%|          |

Downloading:   0%|          |

,num_records
0,1656


In [23]:
%%bigquery
select distinct effective_time, discontinue_time, status_flag
from airline_csp.Air_Carrier

Query is running:   0%|          |

Downloading:   0%|          |

,effective_time,discontinue_time,status_flag
0,2024-02-17 16:27:34.248933+00:00,NaT,True


In [ ]:
%%bigquery
merge airline_csp.Air_Carrier t
using airline_stg.Air_Carrier s
on t.airline_id = s.airline_id
when matched and s.airline_name != t.airline_name or s.airline_code != t.airline_code then
  update set discontinue_time = timestamp_sub(current_timestamp(), interval 1 second), status_flag = false
  insert (airline_id, airline_name, airline_code, data_source, load_time, effective_time, status_flag)
    values (s.airline_id, s.airline_name, s.airline_code, s.data_source, s.load_time, current_timestamp(), true)
when not matched by source then
  update set discontinue_time = current_timestamp(), status_flag = false
when not matched by target then
  insert (airline_id, airline_name, airline_code, data_source, load_time, effective_time, status_flag)
    values (s.airline_id, s.airline_name, s.airline_code, s.data_source, s.load_time, current_timestamp(), true)

Executing query with job ID: e2786a28-e71e-4306-b31d-8ba55449520d
Query executing: 0.24s


ERROR:
 400 Syntax error: Expected end of input but got keyword INSERT at [6:3]

Location: US
Job ID: e2786a28-e71e-4306-b31d-8ba55449520d



#### Unfortunately, BigQuery's merge function does not allow us to place the update and insert statements inside the matched clause. As a result, we will split up the merge operation into two parts.

#### Merge in the new records and the deleted records:

In [24]:
%%bigquery
merge airline_csp.Air_Carrier t
using airline_stg.Air_Carrier s
on t.airline_id = s.airline_id
-- handle deleted records
when not matched by source then
  update set discontinue_time = current_timestamp(), status_flag = false
-- handle new records
when not matched by target then
  insert (airline_id, airline_name, airline_code, data_source, load_time, effective_time, status_flag)
  values (s.airline_id, s.airline_name, s.airline_code, s.data_source, s.load_time, current_timestamp(), true)

Query is running:   0%|          |

""


In [25]:
%%bigquery
select distinct effective_time, discontinue_time, status_flag
from airline_csp.Air_Carrier

Query is running:   0%|          |

Downloading:   0%|          |

,effective_time,discontinue_time,status_flag
0,2024-03-29 16:45:06.061414+00:00,NaT,True
1,2024-02-17 16:27:34.248933+00:00,NaT,True
2,2024-02-17 16:27:34.248933+00:00,2024-03-29 16:45:06.061414+00:00,False


#### Now handle the updated records:

In [26]:
%%bigquery
select s.*
from airline_csp.Air_Carrier t join airline_stg.Air_Carrier s
on t.airline_id = s.airline_id
where s.airline_name != t.airline_name
or s.airline_code != t.airline_code

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,airline_name,airline_code,data_source,load_time
0,20403,Discovery Airways Inc.,DH,bird,2024-03-29 16:35:34.693043+00:00
1,20019,Private Jet Expeditions,5J,bird,2024-03-29 16:35:34.693043+00:00
2,20214,Avia Leasing Company,AD,bird,2024-03-29 16:35:34.693043+00:00
3,19603,Pacific Express,VB,bird,2024-03-29 16:35:34.693043+00:00
4,20306,Challenge Air Transport Inc.,VV,bird,2024-03-29 16:35:34.693043+00:00
...,...,...,...,...,...
104,19986,Turks Air Ltd.,BHQ,bird,2024-03-29 16:35:34.693043+00:00
105,19878,Columbia Pacific Airlines,EXA,bird,2024-03-29 16:35:34.693043+00:00
106,19662,Transbrasil S.A.,TR,bird,2024-03-29 16:35:34.693043+00:00
107,20329,Skystar International Inc.,7S,bird,2024-03-29 16:35:34.693043+00:00


In [27]:
%%bigquery
declare current_ts TIMESTAMP;
set current_ts = current_timestamp();

create temp table updates as
  select s.*
  from airline_csp.Air_Carrier t join airline_stg.Air_Carrier s
  on t.airline_id = s.airline_id
  where s.airline_name != t.airline_name
  or s.airline_code != t.airline_code;

update airline_csp.Air_Carrier
set discontinue_time = timestamp_sub(current_ts, interval 1 second), status_flag = false
where airline_id in (select airline_id from updates);

insert into airline_csp.Air_Carrier
  (airline_id, airline_name, airline_code, data_source, load_time, effective_time, status_flag)
    (select airline_id, airline_name, airline_code, data_source, load_time, current_ts, true
    from updates);

Query is running:   0%|          |

""


In [28]:
%%bigquery
select distinct effective_time, discontinue_time, status_flag
from airline_csp.Air_Carrier

Query is running:   0%|          |

Downloading:   0%|          |

,effective_time,discontinue_time,status_flag
0,2024-03-29 16:45:06.061414+00:00,NaT,True
1,2024-02-17 16:27:34.248933+00:00,NaT,True
2,2024-02-17 16:27:34.248933+00:00,2024-03-29 16:45:06.061414+00:00,False
3,2024-02-17 16:27:34.248933+00:00,2024-03-29 16:45:46.433279+00:00,False
4,2024-03-29 16:45:47.433279+00:00,NaT,True


In [29]:
%%bigquery
select * from airline_csp.Air_Carrier
where airline_id = 20054

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,airline_name,airline_code,data_source,load_time,effective_time,discontinue_time,status_flag
0,20054,Gulf Air Transport Inc.,GF (1),bird,2024-01-26 22:23:22.051288+00:00,2024-02-17 16:27:34.248933+00:00,2024-03-29 16:45:46.433279+00:00,False
1,20054,Gulf Air Transport Inc.,GF,bird,2024-03-29 16:35:34.693043+00:00,2024-03-29 16:45:47.433279+00:00,NaT,True


In [ ]:
%%bigquery
drop table airline_l.air_carriers

Query is running:   0%|          |

""
